# system_simulation


In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from ipywidgets import interact
from extended_kalman_filter import ExtendedKalmanFilter, KalmanFilter

In [ ]:
def plot_util(_plt):
    _plt.legend()
    _plt.grid()
    fig = _plt.gcf();
    fig.set_size_inches(18.5, 5, forward=True)
    _plt.show()

## Load the pre-recorded data

In [ ]:
df_loaded = pd.read_csv('logged_data_1.csv', index_col=0) # or 'logged_data_2.csv'
print('Columns:', [i for i in df_loaded.columns])

phi_yaw_arr = df_loaded.iloc[:,0].values
phi_pitch_arr = df_loaded.iloc[:,1].values
time_arr = df_loaded.iloc[:,2].values

In [ ]:
df_loaded = pd.read_csv('logged_data_3.csv', index_col=0)
print('Columns:', [i for i in df_loaded.columns])

phi_yaw_arr = df_loaded.iloc[:,0].values
phi_pitch_arr = df_loaded.iloc[:,1].values
gc_yaw_arr = df_loaded.iloc[:,2].values
gc_pitch_arr = df_loaded.iloc[:,3].values
time_arr = df_loaded.iloc[:,4].values

nn_updates = []
nn_updates_t = []
prev_val = phi_yaw_arr[0]
for i, val in enumerate(phi_yaw_arr):
    if val != prev_val:
        nn_updates.append(val)
        nn_updates_t.append(time_arr[i])
        prev_val = val

## Sample Kalman Filters

In [ ]:
KF_1 = KalmanFilter(Ts=0.05, Q=0.1,   R=0.1, a=1)
KF_2 = KalmanFilter(Ts=0.05, Q=0.01,  R=0.1, a=1)
KF_3 = KalmanFilter(Ts=0.05, Q=1e-3, R=0.1, a=1)
KF_4 = KalmanFilter(Ts=0.05, Q=1e-3, R=1, a=1)

EKF_1 = ExtendedKalmanFilter(Ts=0.05, Q=0.1,   R=0.1, a=1)
EKF_2 = ExtendedKalmanFilter(Ts=0.05, Q=0.01,  R=0.1, a=1)
EKF_3 = ExtendedKalmanFilter(Ts=0.05, Q=1e-3, R=0.1, a=1)
EKF_4 = ExtendedKalmanFilter(Ts=0.05, Q=1e-3, R=1, a=1)

KFs =  [KF_1, KF_2, KF_3, KF_4]
EKFs = [EKF_1, EKF_2, EKF_3, EKF_4]
labels= ['Q=0.1, R=0.1, a=1']

In [ ]:
results = []
prev_measurement = 0

for KF in EKFs:
    temp_results = []
    for measurement in phi_yaw_arr:
        KF.predict()
        
        if measurement != prev_measurement:
            KF.update(measurement)
            prev_measurement = measurement
        
        temp_results.append(KF.get_pos())
    
    results.append(temp_results)

In [ ]:
plt.stem(nn_updates_t, nn_updates, label='Raw NN updates [deg]')
for i, result in enumerate(results):
    plt.plot(time_arr, result, label='KF_%i' % (i+1))
plt.title('Comparison of KFs with different parameters for Q, R and a')
plot_util(plt)

In [ ]:
# plt.plot(time_arr, EKF_yaw_arr, label='EKF estimate of yaw [deg]')
plt.stem(time_arr, phi_yaw_arr, label='Raw NN estimate of yaw [deg]')
plot_util(plt)

In [ ]:
nn_arr = []
nn_t_arr = []
prev_yaw = 0
for i, yaw in enumerate(phi_yaw_arr):
    if yaw != prev_yaw:
        nn_arr.append(yaw)
        nn_t_arr.append(time_arr[i])
        prev_yaw = yaw

In [ ]:
plt.stem(nn_t_arr, nn_arr, label='Raw NN estimate of yaw [deg]')
plot_util(plt)

In [ ]:
# plt.plot(time_arr, EKF_pitch_arr, label='EKF estimate of pitch [deg]')
plt.plot(time_arr, phi_pitch_arr, label='Raw NN estimate of pitch [deg]')
plot_util(plt)

In [ ]:
loop_times_ms = [(t-t_)*1e3 for t_,t in zip(time_arr[0:-1], time_arr[1:])]
plt.stem(time_arr[:-1], loop_times_ms, label='loop times [ms]')
plot_util(plt)

In [ ]:
def sim_and_plot(model, Q, R, a):
    if model == 'EKF':
        EKF = ExtendedKalmanFilter(Ts=0.05, Q=Q, R=R, a=a, camera_FOV_deg=62.2)
    else:
        EKF = KalmanFilter(Ts=0.05, Q=Q, R=R, a=a)

    data = phi_yaw_arr
    prev_measurement = 0
    results = []
    predict_ahead_results = []
    
    for measurement in data:
#         EKF.predict()
        if measurement != prev_measurement:
            EKF.predict()
            EKF.update(measurement)
            prev_measurement = measurement
        else:
            EKF.predict()
        
        results.append(EKF.get_pos())
        predict_ahead_results.append(EKF.predict_ahead(3)) # 3*Ts = 150 ms

    plt.stem(nn_updates_t, nn_updates, label='Raw NN estimate [deg]')
    plt.plot(time_arr, results, label='EKF estimate [deg]')
#     plt.plot(time_arr, predict_ahead_results, label='EKF estimate predicting 150 ms into future [deg]')
    plot_util(plt)

In [ ]:
# good values for KF: Q = 0.5, R = 0.005, a = 0.94
#  ------------- EKF: Q =   5, R = 0.005, a = 0.95
interact(sim_and_plot, model=['KF', 'EKF'], Q=[5,0.5,0.05], R=[0.5,0.05,0.005], a=[0,0.95,1]);

In [ ]:
EKF = ExtendedKalmanFilter(Ts=0.05, Q=0.5, R=0.005, a=0.94, camera_FOV_deg=62.2)  # values from KF

ret = []
for i in range(100):
    EKF.predict()
    EKF.update(10)  # input = pixels scaled to [-1, 1]
    ret.append(EKF.get_pos())

plt.plot(ret, label='EKF estimate of unit step [-31.1ᵒ, 31.1ᵒ] scaled to [-1, 1]'); plot_util(plt)